Question 5 : Without including the record part

Image stitching function
Stitches based on SIFT or ORB features
Usage :

stitch_2_imgs(img1, img2, feature)

for stitching using ORB features, use "ORB"
for stitching using SIFT features, use "SIFT"

In [5]:
import cv2
import numpy as np

def stitch_2_imgs(img1, img2, feature='SIFT', blending=True):
    # Convert images to grayscale
    img1_bw = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_bw = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    # Choose feature detector based on the provided feature parameter
    if feature == 'SIFT':
        feat = cv2.SIFT_create() 
    elif feature == 'ORB':
        feat = cv2.ORB_create()
    else:
        print("Please enter correct feature value.")
        return None

    # Detect keypoints and compute descriptors for both images
    kp_img1, desc_img1 = feat.detectAndCompute(img1_bw, None) 
    kp_img2, desc_img2 = feat.detectAndCompute(img2_bw, None) 

    # Match descriptors using a brute-force matcher
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(desc_img2, desc_img1, k=2)

    # Filter good matches based on Lowe's ratio test
    good_points = []
    for m, n in matches: 
        if m.distance < 0.6 * n.distance: 
            good_points.append(m) 

    # Check if enough good matches are found to compute homography
    if len(good_points) < 4:
        print("Not enough matches found to compute homography.")
        return None

    # Extract matched keypoints for computing homography
    query_pts = np.float32([kp_img2[m.queryIdx].pt for m in good_points]).reshape(-1, 1, 2) 
    train_pts = np.float32([kp_img1[m.trainIdx].pt for m in good_points]).reshape(-1, 1, 2) 

    # Compute homography matrix using RANSAC
    matrix, mask = cv2.findHomography(query_pts, train_pts, cv2.RANSAC, 5.0) 

    # Warping the second image to the perspective of the first image
    dst = cv2.warpPerspective(img2, matrix, ((img1.shape[1] + img2.shape[1]), img2.shape[0])) 

    # If blending is enabled, perform multi-band blending
    if blending:
        dst[0:img1.shape[0], 0:img1.shape[1]] = img1
    else:
        # Directly paste the second image onto the first image
        dst[0:img1.shape[0], 0:img1.shape[1]] = img1
        # Adjust pixel intensity to avoid overwriting img1 pixels with black pixels from img2
        dst = cv2.addWeighted(dst, 0.5, img2, 0.5, 0)

    return dst


In [4]:
img1 = cv2.imread('image_stitching/building2.jpg')
img2 = cv2.imread('image_stitching/building1.jpg')
result = stitch_2_imgs(img1, img2, blending=True)

if result is not None:
    cv2.imwrite('image_stitching/panorama.jpg', result)
    cv2.imshow('Panorama', result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Stitching failed. Insufficient matches.")
